# Mod13 Handling Missing Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.__version__

'1.19.4'

In [3]:
pd.__version__

'1.1.4'

### Detecting null values

In [45]:
# None in  np.array  => dtype=object
# !　dtype=object　is slow
data00 = np.array([1, None, 3, 4]);data00

array([1, None, 3, 4], dtype=object)

In [44]:
# None in  pd.Serie  => dtype: float64
data01 = pd.Series([1, None, 3, 4]);data01

0    1.0
1    NaN
2    3.0
3    4.0
dtype: float64

In [8]:
# native numpy array
data1 = pd.Series([1, np.nan, 33]);data1

0     1.0
1     NaN
2    33.0
dtype: float64

In [9]:
data1.values

array([ 1., nan, 33.])

In [4]:
# None can only exist in 'object' array
data = pd.Series([1, np.nan, 'hello', None])

In [11]:
pd.DataFrame.cumprod #inplace parameter 原地寫入，但沒有比copy快，可能導致記憶體搬動而溢位，建議避免直接copy


<function pandas.core.frame.DataFrame.cumprod(self, axis=None, skipna=True, *args, **kwargs)>

In [5]:
data # be careful, ==> object

0        1
1      NaN
2    hello
3     None
dtype: object

In [12]:
data.isnull()  # "None" in Pandas is null

0    False
1     True
2    False
3     True
dtype: bool

In [13]:
# Masking 
data[data.notnull()]  # slower than dropna() 

0        1
2    hello
dtype: object

In [14]:
%%timeit
data.isnull().any()

82.5 µs ± 4.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%%timeit
data.isnull().values.any()  # numpy arrray is most fast

56.7 µs ± 5.57 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Dropping null values

For a ``Series``

In [20]:
data.dropna()  # remenber use Series to get

0        1
2    hello
dtype: object

In [21]:
data  = pd.Series(data.dropna());data

0        1
2    hello
dtype: object

For a ``DataFrame``

In [23]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [24]:
df.dropna()  # remove entire row(default)

,0,1,2
1,2.0,3.0,5


In [25]:
df.dropna(axis='columns')  # change default

,2
0,2
1,5
2,6


In [26]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.dropna(axis='columns', how='all')


,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [29]:
# thread: reserve (value != Nan)  >= n。Require that many non-NA values.
df.dropna(axis='rows', thresh=3)   


,0,1,2,3
1,2.0,3.0,5,NaN


# df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)


### Filling null values

#### For ``Series``

fill NA entries with a single value, such as zero:

In [31]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [32]:
# fill with zero
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [33]:
# fill with mean。"Mean()" will not includ NaN
data.fillna(data.mean())

a    1.0
b    2.0
c    2.0
d    2.0
e    3.0
dtype: float64

specify a forward-fill to propagate the previous value forward:

In [34]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [35]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

#### For ``DataFrame``

In [36]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [37]:
type(df.iloc[1])

pandas.core.series.Series

In [39]:
df.fillna(method='ffill', axis=1) 

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [38]:
df.fillna(method='ffill')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN
